In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

In [ ]:
def gaussianCoeff(x, kernelWidth):
    rcpSigma = 3 / kernelWidth
    p = x * rcpSigma
    return np.sqrt(1.0/2/np.pi) * rcpSigma * np.exp(-0.5*p*p)


def cmain():
    k = 23
    x = np.arange(-k, k+1, 1)
    y = gaussianCoeff(x, k)
    plt.bar(x, y)
    plt.show()

    print(y/y.sum())

cmain()

In [ ]:
from matplotlib import cm

def draw_bar3d(data):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #ax.imshow(data, cmap="Greys")
    ax.imshow(data)
    return ax

    fig = plt.figure()
    ax1 = fig.add_subplot(111, projection='3d')

    nx = data.shape[0]
    ny = data.shape[1]
    x = np.arange(0, nx)
    y = np.arange(0, ny)
    x, y = np.meshgrid(x, y)
    ax1.plot_surface(x, y, data)
    return ax1

In [ ]:
def fetch(data, s, t):
    w = data.shape[1]
    h = data.shape[0]
    s = np.minimum(np.maximum(s, 0), w - 1)
    t = np.minimum(np.maximum(t, 0), h - 1)
    lin_data = data.reshape(-1)
    return lin_data[t * w + s]

def sample(data, u, v):
    s = u * data.shape[1] - 0.5
    t = v * data.shape[0] - 0.5

    s0 = s.astype(int)
    ds = s - s0

    t0 = t.astype(int)
    dt = t - t0

    return (1 - ds) * (1 - dt) * fetch(data, s0, t0) + \
           (1 - ds) * dt * fetch(data, s0, t0 + 1) + \
           ds * (1 - dt) * fetch(data, s0 + 1, t0) + \
           ds * dt * fetch(data, s0 + 1, t0 + 1)

def downsample(prev_mip, a_weight):
    w = np.floor(prev_mip.shape[1] * 0.5).astype(int)
    h = np.floor(prev_mip.shape[0] * 0.5).astype(int)

    s = np.arange(0, w)
    t = np.arange(0, h)
    s, t = np.meshgrid(s, t)

    half_pixel_u = 0.5/w
    half_pixel_v = 0.5/h

    u = s / w + half_pixel_u
    v = t / h + half_pixel_v

    accum  = sample(prev_mip, u, v) * a_weight
    accum += sample(prev_mip, u - half_pixel_u, v - half_pixel_v)
    accum += sample(prev_mip, u - half_pixel_u, v + half_pixel_v)
    accum += sample(prev_mip, u + half_pixel_u, v + half_pixel_v)
    accum += sample(prev_mip, u + half_pixel_u, v - half_pixel_v)
    return accum / (4 + a_weight) * 2

def upsample(prev_mip, b_weight, sample_radius):
    w = np.floor(prev_mip.shape[1] * 2).astype(int)
    h = np.floor(prev_mip.shape[0] * 2).astype(int)

    s = np.arange(0, w)
    t = np.arange(0, h)
    s, t = np.meshgrid(s, t)

    half_pixel_u = 0.5/w
    half_pixel_v = 0.5/h

    u = s / w + half_pixel_u
    v = t / h + half_pixel_v

    accum = sample(prev_mip, u, v) * b_weight * 2
    accum += sample(prev_mip, u - half_pixel_u * sample_radius * 2.0, v)
    accum += sample(prev_mip, u - half_pixel_u * sample_radius, v + half_pixel_v * sample_radius) * b_weight
    accum += sample(prev_mip, u, v + half_pixel_v * sample_radius * 2.0)
    accum += sample(prev_mip, u + half_pixel_u * sample_radius, v + half_pixel_v * sample_radius) * b_weight
    accum += sample(prev_mip, u + half_pixel_u * sample_radius * 2.0, v)
    accum += sample(prev_mip, u + half_pixel_u * sample_radius, v - half_pixel_v * sample_radius) * b_weight
    accum += sample(prev_mip, u, v - half_pixel_v * sample_radius * 2.0)
    accum += sample(prev_mip, u - half_pixel_u * sample_radius, v - half_pixel_v * sample_radius) * b_weight
    return  accum / (4 + 6 * b_weight) /4

def bilinear_upsample(prev_mip):
    w = np.floor(prev_mip.shape[1] * 2).astype(int)
    h = np.floor(prev_mip.shape[0] * 2).astype(int)

    s = np.arange(0, w)
    t = np.arange(0, h)
    s, t = np.meshgrid(s, t)

    half_pixel_u = 0.5/w
    half_pixel_v = 0.5/h

    u = s / w + half_pixel_u
    v = t / h + half_pixel_v

    accum = sample(prev_mip, u, v)
    return accum


def kawase_dual_filter(input, iter_cnt, a_weight, b_weight, upsample_radius, plot=False):
    iter_ = iter_cnt
    last_mip = input
    for next_mip_level in np.arange(1, iter_+1):
        tmp = downsample(last_mip, a_weight)
        last_mip = tmp
        # print(last_mip)
        # print(np.sum(last_mip))
        if plot:
            print(last_mip.shape)
            ax = draw_bar3d(last_mip)
            ax.set_title("mip " + str(next_mip_level))
            plt.show()

    prev_mip = last_mip
    iter_ = iter_cnt
    for next_mip_level in np.arange(1, iter_):
        tmp = upsample(prev_mip, b_weight, upsample_radius)
        prev_mip = tmp
        # print(np.sum(prev_mip))
        if plot:
            print(prev_mip.shape)
            ax = draw_bar3d(prev_mip)
            ax.set_title("mip " + str(iter_ - next_mip_level) + ", size ")
            plt.show()

    prev_mip = bilinear_upsample(prev_mip)
    if plot:
        print(prev_mip.shape)
        draw_bar3d(prev_mip).set_title("mip 0")
        plt.show()

    # prev_mip = prev_mip[0: data_1d_gaussian_filtered.shape[0]]
    prev_mip = prev_mip/np.sum(prev_mip)
    return prev_mip

def get_kawase_kernel_size(output_data):
    masked = output_data * (output_data > np.max(output_data) * 0.001).astype(float)
    return np.max(np.count_nonzero(masked, axis=0))

def get_kawase_kernel_size_empirical(iteration_cnt):
    # based on statistics data with upsample radius 2
    lut = [0, 7, 23, 49, 99, 199, 397, 795]
    return lut[iteration_cnt]
    # return 2 ** (iteration_cnt + 2)

def compact_kawase_kernel(filter_kernel):
    w = filter_kernel.shape[1]
    h = filter_kernel.shape[0]
    hw = int(w * 0.5)
    hh = int(h * 0.5)
    kernel_width = get_kawase_kernel_size(filter_kernel)
    kernel_width = int(kernel_width/2)
    return filter_kernel[hh-kernel_width:hh+kernel_width+1, hw-kernel_width:hw+kernel_width+1]

In [ ]:
kernel_radius = 256
kernel_size = kernel_radius * 2
data_impulse = np.zeros([kernel_size, kernel_size])
data_impulse[kernel_radius][kernel_radius] = 1
# draw_bar3d(data_impulse)
# plt.show()

kawase_iter = 6
a_weight = 4
b_weight = 2
upsample_radius = 2

kawase_filtered = kawase_dual_filter(data_impulse, kawase_iter, a_weight, b_weight, upsample_radius, plot=False)
draw_bar3d(kawase_filtered)
plt.show()

kernel_size = get_kawase_kernel_size(kawase_filtered)
print(kernel_size)
print(get_kawase_kernel_size_empirical(kawase_iter))

In [ ]:
kawase_iter = 4
a_weight = 4
b_weight = 2
upsample_radius = 2

kawase_filtered = kawase_dual_filter(data_impulse, kawase_iter, a_weight, b_weight, upsample_radius)
draw_bar3d(kawase_filtered)
plt.show()

# kawase_filtered_compact = compact_kawase_kernel(kawase_filtered)
# draw_bar3d(kawase_filtered_compact)
# plt.show()

kernel_size = get_kawase_kernel_size(kawase_filtered)
print(kernel_size)
print(get_kawase_kernel_size_empirical(kawase_iter))

gaussian_kernel_radius = get_kawase_kernel_size_empirical(kawase_iter) / 2
print(gaussian_kernel_radius)
gaussian_filter_x = gaussianCoeff(np.arange(-gaussian_kernel_radius, gaussian_kernel_radius+1), gaussian_kernel_radius)
gaussian_filter_y = gaussianCoeff(np.arange(-gaussian_kernel_radius, gaussian_kernel_radius+1), gaussian_kernel_radius)
gaussian_filter_x, gaussian_filter_y = np.meshgrid(gaussian_filter_x, gaussian_filter_y)
gaussian_filter = gaussian_filter_x * gaussian_filter_y
gaussian_filter /= np.sum(gaussian_filter)
# draw_bar3d(gaussian_filter)
# plt.show()

gaussian_filtered_ = convolve2d(data_impulse, gaussian_filter, mode="same")
# gaussian_filtered_ = gaussian_filtered_[kernel_radius - gaussian_kernel_radius: kernel_radius + gaussian_kernel_radius, kernel_radius - gaussian_kernel_radius : kernel_radius + gaussian_kernel_radius]
draw_bar3d(gaussian_filtered_)
plt.show()

In [ ]:
get_kawase_kernel_size(gaussian_filtered_)

In [ ]:
from scipy import optimize

def loss(params):
    a_weight, b_weight, upsample_radius = params
    kawase_filtered = kawase_dual_filter(data_impulse, kawase_iter, a_weight, b_weight, upsample_radius)
    shape_offset = ((np.array(kawase_filtered.shape) - np.array(gaussian_filtered_.shape)) * 0.5).astype(int)
    kawase_filtered = kawase_filtered[shape_offset[0]: gaussian_filtered_.shape[0] + shape_offset[0],
                                      shape_offset[1]: gaussian_filtered_.shape[1] + shape_offset[1]]
    diff = kawase_filtered - gaussian_filtered_
    return np.mean(np.abs(diff)**4)

initial_guess = [4,2,2]
result = optimize.minimize(loss, initial_guess, method="Nelder-Mead")
print(result)
# plt.plot(x, y, label="exact")
# plt.plot(x, y_, label="approx")
# plt.legend()
# plt.show()



In [ ]:

approx = kawase_dual_filter(data_impulse, kawase_iter, result.x[0], result.x[1], result.x[2])
draw_bar3d(approx)
plt.show()

draw_bar3d(gaussian_filtered_)
plt.show()

approx_compact = compact_kawase_kernel(approx)
ax = draw_bar3d(approx_compact)
ax.set_title("kawase dual")
plt.show()

ax = draw_bar3d(gaussian_filter)
ax.set_title("gaussian")
plt.show()

diff = np.abs(compact_kawase_kernel(approx - gaussian_filtered_))
ax = draw_bar3d(diff)
ax.set_zlim(0,np.max(gaussian_filter))
ax.set_title("diff")
plt.show()


In [ ]:
get_kawase_kernel_size_empirical(kawase_iter)

In [ ]:
[7.0242585 , 1.52714585, 2.44185195]
[5.69430406, 1.40275451, 2.37575196]
[5.1570802 , 1.53785764, 2.37161158]
[4.88042859, 1.69690314, 2.40855495]
